In [5]:
from datasets import load_dataset

cc_news = load_dataset("cc_news")

Dataset cc_news downloaded and prepared to /Users/mismayil/.cache/huggingface/datasets/cc_news/plain_text/1.0.0/ae469e556251e6e7e20a789f93803c7de19d0c4311b6854ab072fecb4e401bd6. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00,  4.06it/s]


In [15]:
from torch.utils.data import DataLoader

loader = DataLoader(cc_news["train"], shuffle=True, batch_size=100)

In [16]:
samples = next(iter(loader))

len(samples["description"])

100

In [17]:
with open("data/cc_news_samples.txt", "w") as f:
    f.write("\n".join([sample for sample in samples["description"] if sample.strip()]))

In [18]:
daily_dialog = load_dataset("daily_dialog", split="test")

Dataset daily_dialog downloaded and prepared to /Users/mismayil/.cache/huggingface/datasets/daily_dialog/default/1.0.0/1d0a58c7f2a4dab5ed9d01dbde8e55e0058e589ab81fce5c2df929ea810eabcd. Subsequent calls will reuse this data.


In [19]:
daily_dialog

Dataset({
    features: ['dialog', 'act', 'emotion'],
    num_rows: 1000
})

In [22]:
loader = DataLoader(daily_dialog, shuffle=True, batch_size=50)

In [29]:
daily_dialog[:2]

{'dialog': [['Hey man , you wanna buy some weed ? ',
   ' Some what ? ',
   ' Weed ! You know ? Pot , Ganja , Mary Jane some chronic ! ',
   ' Oh , umm , no thanks . ',
   ' I also have blow if you prefer to do a few lines . ',
   ' No , I am ok , really . ',
   ' Come on man ! I even got dope and acid ! Try some ! ',
   ' Do you really have all of these drugs ? Where do you get them from ? ',
   ' I got my connections ! Just tell me what you want and I ’ ll even give you one ounce for free . ',
   ' Sounds good ! Let ’ s see , I want . ',
   ' Yeah ? ',
   ' I want you to put your hands behind your head ! You are under arrest ! '],
  ['The taxi drivers are on strike again . ',
   ' What for ? ',
   ' They want the government to reduce the price of the gasoline . ',
   ' It is really a hot potato . ']],
 'act': [[3, 2, 3, 4, 3, 4, 3, 2, 3, 4, 2, 3], [1, 2, 1, 1]],
 'emotion': [[0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0], [0, 0, 0, 0]]}

In [30]:
dialog_samples = ["".join(dialog) for dialog in daily_dialog[:50]["dialog"]]

In [31]:
len(dialog_samples)

50

In [32]:
with open("data/dialog_samples.txt", "w") as f:
    f.write("\n".join([sample for sample in dialog_samples if sample.strip()]))

In [33]:
roc_stories = load_dataset("wza/roc_stories")

No config specified, defaulting to: roc_stories/all


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 320.00it/s]


Dataset roc_stories downloaded and prepared to /Users/mismayil/.cache/huggingface/datasets/wza___roc_stories/all/2.1.0/43e2851d9f31e08e4b2dd07a8057ed7a64cbb25cc7105d09856c14e638695506. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 113.57it/s]


In [34]:
roc_stories

DatasetDict({
    train: Dataset({
        features: ['storyid', 'storytitle', 'sentence1', 'sentence2', 'sentence3', 'sentence4', 'sentence5'],
        num_rows: 98161
    })
})

In [37]:
roc_samples = roc_stories["train"][:50]
roc_samples = [" ".join(sample) for sample in zip(roc_samples["sentence1"], roc_samples["sentence2"], roc_samples["sentence3"], roc_samples["sentence4"], roc_samples["sentence5"])]

In [38]:
roc_samples[:3]

["Dan's parents were overweight. Dan was overweight as well. The doctors told his parents it was unhealthy. His parents understood and decided to make a change. They got themselves and Dan on a diet.",
 "Carrie had just learned how to ride a bike. She didn't have a bike of her own. Carrie would sneak rides on her sister's bike. She got nervous on a hill and crashed into a wall. The bike frame bent and Carrie got a deep gash on her leg.",
 "Morgan enjoyed long walks on the beach. She and her boyfriend decided to go for a long walk. After walking for over a mile, something happened. Morgan decided to propose to her boyfriend. Her boyfriend was upset he didn't propose to her first."]

In [ ]:
with open("data/story_samples.txt", "w") as f:
    f.write("\n".join([sample for sample in roc_samples if sample.strip()]))

In [3]:
import spacy

text = "PersonX falls in love with PersonY"
nlp = spacy.load("en_core_web_sm")
doc = nlp(text)

for token in doc:
    print(token.text, token.dep_, [child for child in token.children])

PersonX nsubj []
falls ROOT [PersonX, in, with]
in prep [love]
love pobj []
with prep [PersonY]
PersonY pobj []


In [4]:
import string
", short conversation   ".strip(string.punctuation+" ")

'short conversation'

In [2]:
from kogito.core.processors.head import NounPhraseHeadExtractor, VerbPhraseHeadExtractor
import spacy

nlp = spacy.load("en_core_web_sm")

ext = NounPhraseHeadExtractor("ext", nlp)
vext = VerbPhraseHeadExtractor("vext", nlp)
ext.extract("April 24 (Reuters) - Borrowing by U.S. businesses for capital goods rose 2 percent in March from a year earlier, the Equipment Leasing and Finance")

[Borrowing,
 businesses,
 capital,
 goods,
 percent,
 year,
 Reuters,
 U.S. businesses,
 capital goods,
 March]

In [3]:
from kogito.inference import CommonsenseInference

# Initialize inference module with a spacy language pipeline
csi = CommonsenseInference(language="en_core_web_sm")

# Run inference
text = "the researcher"
csi.remove_processor("noun_phrase_extractor")
csi.remove_processor("verb_phrase_extractor")
csi.remove_processor("sentence_extractor")
kgraph = csi.infer(text, extract_heads=True)
print(kgraph)
# Save output knowledge graph to JSON file
# kgraph.to_jsonl("results/test.json")

Extracting knowledge heads...
None


/Users/mismayil/Desktop/EPFL/nlplab/kogito/kogito/inference.py:168: UserWarning: Skipping inference, no heads found.
  warnings.warn("Skipping inference, no heads found.")


In [10]:
from kogito.inference import CommonsenseInference
from kogito.core.knowledge import KnowledgeGraph
from kogito.models.gpt3.zeroshot import GPT3Zeroshot
from kogito.core.relation import KnowledgeRelation, register_relation, EVENT_RELATIONS
import time, os

# csi = CommonsenseInference()
# csi.remove_processor("simple_relation_matcher")

def x_wishes_verbalizer(head, **kwargs):
   # index will be passed from the model
   # so that we can enumerate samples which helps with inference
   index = kwargs.get("index")
   index_txt = f"{index}" if index is not None else ""
   return f"Situation {index_txt}: {head}\nWishes: As a result, PersonX wishes"

X_WISHES = KnowledgeRelation("xWishes",
                             verbalizer=x_wishes_verbalizer,
                             prompt="How does this situation affect each character's wishes?")
register_relation(X_WISHES, kind="social")
print(X_WISHES in EVENT_RELATIONS)

True


In [1]:
from kogito.core.knowledge import Knowledge, KnowledgeGraph

kg1 = Knowledge("h1", "xWants", ["hi", "", "  "])
kg2 = Knowledge("h2", "xWants", ["hi", "ho", " hah "])
kg3 = Knowledge("h3", "xWants", ["hi", "", " huha "])
kg4 = Knowledge("h4", "xWants", ["", " ", "  "])

kgraph = KnowledgeGraph([kg1, kg2, kg3, kg4])
print(kgraph)
kgraph.clean()
print(kgraph)

/Users/mismayil/.pyenv/versions/3.8.12/envs/kogito/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Knowledge(head="h1", relation="xWants", tails=['hi', '', '  '])
Knowledge(head="h2", relation="xWants", tails=['hi', 'ho', ' hah '])
Knowledge(head="h3", relation="xWants", tails=['hi', '', ' huha '])
Knowledge(head="h4", relation="xWants", tails=['', ' ', '  '])
Knowledge(head="h1", relation="xWants", tails=['hi'])
Knowledge(head="h2", relation="xWants", tails=['hi', 'ho', 'hah'])
Knowledge(head="h3", relation="xWants", tails=['hi', 'huha'])


In [4]:
from kogito.core.utils import SortishSampler

ss = SortishSampler(list(range(100)), 16)

for s in ss:
    print(s)

99
98
97
96
95
94
93
92
91
90
89
88
87
86
85
84
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
67
66
65
64
63
62
61
60
59
58
57
56
55
54
53
52
83
82
81
80
79
78
77
76
75
74
73
72
71
70
69
68
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
51
50
49
48
47
46
45
44
43
42
41
40
39
38
37
36
3
2
1
0


In [1]:
from transformers import BertModel

bert = BertModel.from_pretrained("bert-base-uncased")
print(bert)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [3]:
from transformers import pipeline

extractor = pipeline("feature-extraction", model="bert-base-uncased")
output = extractor("Hello world", return_tensors="pt")

print(output.shape)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([1, 4, 768])


In [4]:
output

tensor([[[-0.1689,  0.1361, -0.1394,  ..., -0.6251,  0.0522,  0.3671],
         [-0.3633,  0.1412,  0.8800,  ...,  0.1043,  0.2888,  0.3727],
         [-0.6986, -0.6988,  0.0645,  ..., -0.2210,  0.0099, -0.5940],
         [ 0.8310,  0.1237, -0.1512,  ...,  0.1031, -0.6779, -0.2629]]])

In [5]:
from transformers import BertLMHeadModel

bert = BertLMHeadModel.from_pretrained("bert-base-uncased")
print(bert)

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertLMHeadModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [6]:
from transformers import BartForConditionalGeneration

bart = BartForConditionalGeneration.from_pretrained("facebook/bart-base")
print(bart)

Downloading: 100%|██████████| 1.72k/1.72k [00:00<00:00, 564kB/s]
Downloading: 100%|██████████| 558M/558M [00:28<00:00, 19.8MB/s] 


BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05,